<ul>
<li>Provide a list of audiences activated each quarter, broken down by vehicle.</li>
<li>Include details about the activation path, such as direct vs. NS vs. LR, and the use of Permutive.</li>
<li>Describe the nature of the audience, e.g., whether it's 3p (third-party) or 1p (first-party), etc.</li>
<li>Additionally, please specify the associated media spend for each audience.</li>
<li>Include information about the funnel stage, including segments like high propensity, active, etc.</li>
<li>Present performance results, with a preference for comparing changes to baseline data if it is available.</li>
<ul>

<h2>Library</h2>

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# Function to create node dictionary
def create_node_dict(column):
    unique_values = df[column].unique().tolist()
    node_dict = dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=unique_values,
    )
    return node_dict

# Function to create link dictionary
def create_link_dict(source_column, target_column, values_column):
    source_values = df[source_column].map({v: i for i, v in enumerate(df[source_column].unique())}).tolist()
    target_values = df[target_column].map({v: i + len(df[source_column].unique()) for i, v in enumerate(df[target_column].unique())}).tolist()
    values = df[values_column].tolist()
    link_dict = dict(
        source=source_values + target_values,
        target=target_values,
        value=values,
    )
    return link_dict


<h2>Main</h2>

In [ ]:
df = pd.read_csv('csv/Audience Report Formatted.csv')


In [ ]:
df_audiences = df[['Country','Dell Fiscal Quarter','Audience ID','Spend','Revenue']]

df_audiences = df_audiences.groupby(['Audience ID','Country','Dell Fiscal Quarter',])[['Spend','Revenue']].sum().reset_index()

In [ ]:
df_audiences.head()

In [ ]:
# Create a new column 'Status' to initially mark all segments as 'New'
df_audiences['Status'] = 'New'

# Iterate through each unique 'Segment ID'
for audience_id in df_audiences['Audience ID'].unique():
    # Find the rows where 'Segment ID' matches the current ID
    segment_rows = df_audiences[df_audiences['Audience ID'] == audience_id]
    
    # Check if there is more than one row for this segment (i.e., it was reused)
    if len(segment_rows) > 1:
        # Set 'Status' to 'Reused' for all rows except the first one
        df_audiences.loc[segment_rows.index[1:], 'Status'] = 'Reused'

In [ ]:
pivot_df = df_audiences.pivot_table(index=['Audience ID','Status'], columns='Dell Fiscal Quarter', values='Spend', aggfunc='count')

pivot_df

In [ ]:
df_audiences.to_csv('csv/df_audiences.csv')

In [ ]:
df = df.groupby(['Country', '1/2/3 PD', 'Audience Type Category', 'Vehicle'])['Spend'].sum().reset_index()

df.to_csv('csv/to_import.csv')

In [ ]:
# Create a Sankey diagram
fig = go.Figure(go.Sankey(
    node=create_node_dict('Country'),
    link=create_link_dict('Country', '1/2/3 PD', 'Spend'),
))

# Add more nodes and links
fig.add_trace(go.Sankey(
    node=create_node_dict('1/2/3 PD'),
    link=create_link_dict('1/2/3 PD', 'Audience Type Category', 'Spend'),
))

fig.add_trace(go.Sankey(
    node=create_node_dict('Audience Type Category'),
    link=create_link_dict('Audience Type Category', 'Vehicle', 'Spend'),
))

# Customize the Sankey diagram appearance (optional)
fig.update_layout(
    title_text="Sankey Diagram Example",
    font_size=10,
    hovermode="x",
)

# Display the Sankey diagram
fig.show()

In [ ]:
import plotly.graph_objects as go

# Define the source, target, and value for a single flow
source = [0]  # Index of the source node
target = [1]  # Index of the target node
value = [10]  # Flow value

# Create a simple Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,         # Padding between nodes
        thickness=20,   # Thickness of the node rectangles
        line=dict(color="black", width=0.5),  # Node border color and width
        label=["Source", "Target"],  # Node labels
    ),
    link=dict(
        source=source,
        target=target,
        value=value,
    )
))

# Display the Sankey diagram
fig.show()
